In [1]:
import sys
sys.path.append('../')

from IPython.core.display import display
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("./movie_ratings/movie_ratings.csv")

In [3]:
display(df.head())
display(df.describe())

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


,userId,movieId,rating,timestamp
count,100004.000000,100004.000000,100004.000000,1.000040e+05
mean,347.011310,12548.664363,3.543608,1.129639e+09
std,195.163838,26369.198969,1.058064,1.916858e+08
min,1.000000,1.000000,0.500000,7.896520e+08
25%,182.000000,1028.000000,3.000000,9.658478e+08
50%,367.000000,2406.500000,4.000000,1.110422e+09
75%,520.000000,5418.000000,4.000000,1.296192e+09
max,671.000000,163949.000000,5.000000,1.476641e+09


In [4]:
# make user/movie ids unique
num_users = len(df['userId'].value_counts())
df['movieId'] = df['movieId'] + num_users + 329

display(df.head())
display(df.describe())

,userId,movieId,rating,timestamp
0,1,1031,2.5,1260759144
1,1,2029,3.0,1260759179
2,1,2061,3.0,1260759182
3,1,2129,2.0,1260759185
4,1,2172,4.0,1260759205


,userId,movieId,rating,timestamp
count,100004.000000,100004.000000,100004.000000,1.000040e+05
mean,347.011310,13548.664363,3.543608,1.129639e+09
std,195.163838,26369.198969,1.058064,1.916858e+08
min,1.000000,1001.000000,0.500000,7.896520e+08
25%,182.000000,2028.000000,3.000000,9.658478e+08
50%,367.000000,3406.500000,4.000000,1.110422e+09
75%,520.000000,6418.000000,4.000000,1.296192e+09
max,671.000000,164949.000000,5.000000,1.476641e+09


In [5]:
# export movie_ratings_unique with unique user/movie ids
df.to_csv("./movie_ratings/movie_ratings_unique.csv")

In [6]:
# filter to likes (rating ≥ 5)
df = df[df.rating >= 5]

In [7]:
display(df.head())
display(df.userId.value_counts())

,userId,movieId,rating,timestamp
21,2,1017,5.0,835355681
22,2,1039,5.0,835355604
29,2,1150,5.0,835355395
37,2,1222,5.0,835355840
44,2,1265,5.0,835355697


564    408
232    243
242    219
547    214
30     196
      ... 
347      1
455      1
335      1
135      1
122      1
Name: userId, Length: 644, dtype: int64

In [8]:
# get max_rating for user
df = df.join(df.groupby('userId')['rating'].max().rename('user_max_rating'), on='userId')
df

,userId,movieId,rating,timestamp,user_max_rating
21,2,1017,5.0,835355681,5.0
22,2,1039,5.0,835355604,5.0
29,2,1150,5.0,835355395,5.0
37,2,1222,5.0,835355840,5.0
44,2,1265,5.0,835355697,5.0
...,...,...,...,...,...
99975,671,5306,5.0,1064245548,5.0
99978,671,5886,5.0,1064245488,5.0
99979,671,5896,5.0,1065111996,5.0
99982,671,5993,5.0,1064245483,5.0


In [9]:
# for each user get min movieId with highest rating => testMovies
testMovies = df[df['rating'] == df['user_max_rating']].groupby('userId')['movieId'].min().rename('test_movie')
# mark test movies
df = df.join(testMovies, on='userId')
df['is_test'] = df['movieId'] == df['test_movie']
# drop tmp columns
df.drop(['user_max_rating', 'test_movie'], axis=1, inplace=True)

df

,userId,movieId,rating,timestamp,is_test
21,2,1017,5.0,835355681,True
22,2,1039,5.0,835355604,False
29,2,1150,5.0,835355395,False
37,2,1222,5.0,835355840,False
44,2,1265,5.0,835355697,False
...,...,...,...,...,...
99975,671,5306,5.0,1064245548,False
99978,671,5886,5.0,1064245488,False
99979,671,5896,5.0,1065111996,False
99982,671,5993,5.0,1064245483,False


In [10]:
train = df[df['is_test'] == False]
display(train.head())
display(train.describe())

,userId,movieId,rating,timestamp,is_test
22,2,1039,5.0,835355604,False
29,2,1150,5.0,835355395,False
37,2,1222,5.0,835355840,False
44,2,1265,5.0,835355697,False
45,2,1266,5.0,835355586,False


,userId,movieId,rating,timestamp
count,14451.000000,14451.000000,14451.0,1.445100e+04
mean,349.220123,10364.127465,5.0,1.081178e+09
std,194.496481,23341.503697,0.0,1.950448e+08
min,2.000000,1002.000000,5.0,7.896520e+08
25%,187.000000,1918.500000,5.0,9.472925e+08
50%,363.000000,2704.000000,5.0,9.862436e+08
75%,530.000000,4702.000000,5.0,1.225364e+09
max,671.000000,164949.000000,5.0,1.476623e+09


In [11]:
test = df[df['is_test']]
display(test.head())
display(test.describe())

,userId,movieId,rating,timestamp,is_test
21,2,1017,5.0,835355681,True
101,3,1318,5.0,1298862121,True
148,4,1034,5.0,949919556,True
368,5,1597,5.0,1163373711,True
454,6,1293,5.0,1108134539,True


,userId,movieId,rating,timestamp
count,644.000000,644.000000,644.0,6.440000e+02
mean,336.389752,2071.281056,5.0,1.095830e+09
std,194.559922,7144.029441,0.0,2.123092e+08
min,2.000000,1001.000000,5.0,8.282124e+08
25%,166.750000,1025.000000,5.0,9.372049e+08
50%,336.500000,1111.000000,5.0,1.046702e+09
75%,505.250000,1400.500000,5.0,1.281543e+09
max,671.000000,136887.000000,5.0,1.476478e+09


In [12]:
# write train/test datasets
train.to_csv("../data/movie_ratings/movie_ratings.csv")
train.to_csv("./movie_ratings/movie_ratings_train.csv")
test.to_csv("./movie_ratings/movie_ratings_test.csv")

print("done")

done
